## Generating Synthetic Data

*(this notebook is designed to be run in Kaggle)*

In [9]:
! rm -r fake-news fake_news
! git clone -b feature/compare-classif --single-branch https://github.com/massimo-ml/fake-news.git
# ! cp -r ./fake-news/fake_news .

rm: cannot remove 'fake-news': No such file or directory
rm: cannot remove 'fake_news': No such file or directory
Cloning into 'fake-news'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 244 (delta 27), reused 34 (delta 15), pack-reused 181
Receiving objects: 100% (244/244), 131.27 MiB | 24.31 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Updating files: 100% (37/37), done.


In [1]:
! pip install -r fake-news/requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
import pandas as pd

from tqdm import tqdm
import sys
sys.path.append("/kaggle/working/fake-news")

from fake_news.generators import Phi2Generator, TinyLlamaGenerator, T5Generator

In [3]:
# generator, save_path = TinyLlamaGenerator("AmevinLS/tinyllama-lora-realnews"), "./generated/tinyllama_real_articles.csv"
# generator, save_path = Phi2Generator("AmevinLS/phi-2-lora-realnews"), "./generated/phi2_real_articles.csv"
# generator, save_path = T5Generator("AmevinLS/flan-t5-base-realnews"), "./generated/flant5_real_articles.csv"

# generator, save_path = TinyLlamaGenerator("AmevinLS/tinyllama-lora-fakenews"), "./generated/tinyllama_fake_articles.csv"
# generator, save_path = Phi2Generator("AmevinLS/phi-2-lora-fakenews"), "./generated/phi2_fake_articles.csv"
generator, save_path = T5Generator("AmevinLS/flan-t5-base-fakenews"), "./generated/flant5_fake_articles.csv"

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
train_df = pd.read_csv("../input/welfake-clean/WELFake_clean_train.csv")

article_lengths = train_df["text"].str.split().str.len()
long_train_df = train_df[article_lengths > 200]

titles_for_generation = long_train_df["title"].sample(2000, random_state=42).to_list()
titles_for_generation[:20]

[' Robert DeNiro Axes Anti-Vaxxer Movie From Tribeca Film Festival',
 'U.S. lawmakers want to restrict internet surveillance on Americans',
 'Japan, U.S., South Korea to hold missile tracking drill amid North Korea crisis',
 'China sentences Taiwanese activist to five years for subversion',
 'Governor extends Flint water emergency as state approves funds',
 'A Long Road to the Epigenome',
 "Obama's Attorney General Warned FBI Director Not to Inform Congress of New Hillary Investigation",
 'Put Some Me-ing in Your Be-ing',
 'Silent Counter-Coup by 17 Intel Agencies To Stop Crooked Clintons [Video]',
 ' Duggar-Linked Ministry Building A ‘Retreat’ Where Christian Dads Can Trade Their Teen Daughters',
 'BREAKING: Fresno Police Release GRAPHIC Video Of FATAL SHOOTING OF UNARMED WHITE MAN By Veteran Cops',
 'EPA chief visits Indiana waste site amid proposed budget cuts',
 'Rappoport: ‘CNN Already Deflecting From the Susan Rice Scandal’',
 'RACE-BAITING COP HATERS Dealt Major Blow: Baltimore 

In [5]:
batch_size = 100
all_articles = []
for start_ind in tqdm(range(0, len(titles_for_generation), batch_size)):
    curr_titles = titles_for_generation[start_ind : start_ind+batch_size]
    curr_articles = generator.generate(
        titles=curr_titles, 
        generation_config={
            "max_new_tokens": 200,
            "repetition_penalty": 2.0
        }
    )
    all_articles.extend(curr_articles)

  0%|          | 0/20 [00:00<?, ?it/s]2024-06-12 22:23:09.464558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 22:23:09.464700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 22:23:09.599512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 20/20 [03:41<00:00, 11.09s/it]


In [6]:
synth_df = pd.DataFrame()
synth_df["title"] = titles_for_generation
synth_df["text"] = all_articles
synth_df["label"] = 0

synth_df.to_csv(save_path, index=False)

In [7]:
synth_df

,title,text,label
0,Robert DeNiro Axes Anti-Vaxxer Movie From Tri...,AXEs Anti-Vaxxer Movie From Tribeca Film Festi...,0
1,U.S. lawmakers want to restrict internet surve...,ACR s report on what they believe is the great...,0
2,"Japan, U.S., South Korea to hold missile track...",Athletes such as Satish Sinha and Ashraf Shafi...,0
3,China sentences Taiwanese activist to five yea...,Appleseed Another Taiwanese activist has been ...,0
4,Governor extends Flint water emergency as stat...,Email Proofreader Maine State Rep. Mike Pompeo...,0
...,...,...,...
1995,U.S. Commerce chief warns against China semico...,The information contained herein is for educat...,0
1996,Flap in Congress over gender identity sinks sp...,A clash between Democrats and Republicans over...,0
1997,HEY RACHEL MADDOW…While You’re On The Subject ...,Everything you ever wanted to know about the t...,0
1998,Fed lowers economic outlook but stays mum on r...,The Fed is expected to unveil a new economic o...,0
